<a href="https://colab.research.google.com/github/RutujaJadhav/visualqna/blob/master/visualqnq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visual Question Answering Demo and Tutorial


## Load the libraries

In [0]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import os, argparse
import cv2, spacy, numpy as np
from keras.models import model_from_json
from keras.optimizers import SGD
from sklearn.externals import joblib
from keras import backend as K
from keras.utils.vis_utils import plot_model
K.set_image_data_format('channels_first')
#K.set_image_dim_ordering('th')

Using TensorFlow backend.


## Load the models and weights files

Here is a link to the needed weights

In [0]:
# File paths for the model, all of these except the CNN Weights are 
# provided in the repo, See the models/CNN/README.md to download VGG weights
VQA_model_file_name      = 'models/VQA/VQA_MODEL.json'
VQA_weights_file_name   = 'models/VQA/VQA_MODEL_WEIGHTS.hdf5'
label_encoder_file_name  = 'models/VQA/FULL_labelencoder_trainval.pkl'
CNN_weights_file_name   = 'models/CNN/vgg16_weights.h5'


## Model Idea
Classical CNN-LSTM structure 


<img src="http://i.imgur.com/Za5P1ZZ.png">
[Source](http://arxiv.org/pdf/1505.00468v4.pdf)

# Image features 

## Pretrained VGG Net (VGG-16)

While VGG Net is not the best CNN model for image features, GoogLeNet (winner 2014) and ResNet (winner 2015) have superior classification scores, but VGG Net is very versatile, simple, relatively small and more importantly portable to use. 

For reference here is the VGG 16 performance on ILSVRC-2012
<img src="http://www.robots.ox.ac.uk/~vgg/research/very_deep/images/table_ILSVRC.png">

## Compile the model

In [0]:
def get_image_model(CNN_weights_file_name):
    ''' Takes the CNN weights file, and returns the VGG model update 
    with the weights. Requires the file VGG.py inside models/CNN '''
    from models.CNN.VGG import VGG_16
    image_model = VGG_16(CNN_weights_file_name)
    image_model.layers.pop()
    image_model.layers.pop()
    # this is standard VGG 16 without the last two layers
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    # one may experiment with "adam" optimizer, but the loss function for
    # this kind of task is pretty standard
    image_model.compile(optimizer=sgd, loss='categorical_crossentropy')
    return image_model

## Plot the Model

Keras has a function which allows you to visualize the model in block diagram. Let's do it !

In [0]:
model_vgg = get_image_model(CNN_weights_file_name)
plot_model(model_vgg, to_file='model_vgg.png')

<img src="https://github.com/iamaaditya/VQA_Demo/blob/master/model_vgg.png?raw=1">

## Extract Image features and word embeddings


Extracting image features involves, taking a raw image, and running it through the model, until we reach the last layer. In this case our model is not 100% same as VGG Net, because we are not going to use the last two layer of the VGG. It is because the last layer of VGG Net is a 1000 way softmax and the second last layer is the Dropout.

Thus we are extracting the 4096 Dimension image features from VGG-16


In [0]:
def get_image_features(image_file_name):
    ''' Runs the given image_file to VGG 16 model and returns the 
    weights (filters) as a 1, 4096 dimension vector '''
    image_features = np.zeros((1, 4096))
    # Magic_Number = 4096  > Comes from last layer of VGG Model

    # Since VGG was trained as a image of 224x224, every new image
    # is required to go through the same transformation
    im = cv2.resize(cv2.imread(image_file_name), (224, 224))
    im = im.transpose((2,0,1)) # convert the image to RGBA

    
    # this axis dimension is required because VGG was trained on a dimension
    # of 1, 3, 224, 224 (first axis is for the batch size
    # even though we are using only one image, we have to keep the dimensions consistent
    im = np.expand_dims(im, axis=0) 

    image_features[0,:] = model_vgg.predict(im)[0]
    return image_features
  
##word embeddings
  
def get_question_features(question):
    ''' For a given question, a unicode string, returns the time series vector
    with each word (token) transformed into a 300 dimension representation
    calculated using Glove Vector '''
    word_embeddings = spacy.load('en_vectors_web_lg')
    tokens = word_embeddings(question)
    question_tensor = np.zeros((1, 30, 300))
    for j in xrange(len(tokens)):
        question_tensor[0,j,:] = tokens[j].vector
    return question_tensor

## VQA Model

VQA is a simple model which combines features from Image and Word Embeddings and runs a multiple layer perceptron.

In [0]:
def get_VQA_model(VQA_model_file_name, VQA_weights_file_name):
    ''' Given the VQA model and its weights, compiles and returns the model '''

    # thanks the keras function for loading a model from JSON, this becomes
    # very easy to understand and work. Alternative would be to load model
    # from binary like cPickle but then model would be obfuscated to users
    vqa_model = model_from_json(open(VQA_model_file_name).read())
    vqa_model.load_weights(VQA_weights_file_name)
    vqa_model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return vqa_model

In [0]:
model_vqa = get_VQA_model(VQA_model_file_name, VQA_weights_file_name)
plot_model(model_vqa, to_file='model_vqa.png')

<img src="https://github.com/iamaaditya/VQA_Demo/blob/master/model_vqa.png?raw=1">

As it can be seen above the model also runs a 3 layered LSTM on the word embeddings. To get a naive result it is sufficient to feed the word embeddings directly to the merge layer, but as mentioned above model is gives close to the state-of-the-art results.

Also, four layers of fully connected layers might not be required to achieve a good enough results. But I settled on this model after some experimentation, and their results beat few layers. 

## Ask your question

Let's give a test image and a question

In [0]:
image_file_name = 'test.jpg'
question = u"What vehicle is in the picture?"

# <center> What vehicle is in the picture ? </center>

<img src="https://github.com/iamaaditya/VQA_Demo/blob/master/test.jpg?raw=1">

In [0]:
# get the image features
image_features = get_image_features(image_file_name)

In [0]:
# get the question features
question_features = get_question_features(question)

In [0]:
y_output = model_vqa.predict([question_features, image_features])

# This task here is represented as a classification into a 1000 top answers
# this means some of the answers were not part of training and thus would 
# not show up in the result.
# These 1000 answers are stored in the sklearn Encoder class
warnings.filterwarnings("ignore", category=DeprecationWarning)
labelencoder = joblib.load(label_encoder_file_name)
for label in reversed(np.argsort(y_output)[0,-5:]):
    print str(round(y_output[0,label]*100,2)).zfill(5), "% ", labelencoder.inverse_transform(label)

78.72 %  yes
21.28 %  no
000.0 %  girl
000.0 %  halloween
000.0 %  left


## Results
I am copying the output of the previous command, so that you can validate if your results are same as mine.

**
51.87 %  train <br/>
031.5 %  bicycle <br/>
03.81 %  bike <br/>
02.91 %  bus <br/>
02.54 %  scooter <br/>
**

# Demo with image URL

Since cv2.imread cannot read an image from URL we will have to change our function `get_image_features`

In [0]:
def get_image_features(image_file_name):
    ''' Runs the given image_file to VGG 16 model and returns the 
    weights (filters) as a 1, 4096 dimension vector '''
    image_features = np.zeros((1, 4096))
        
    from skimage import io
    # if you would rather not install skimage, then use cv2.VideoCapture which surprisingly can read from url
    # see this SO answer http://answers.opencv.org/question/16385/cv2imread-a-url/?answer=16389#post-id-16389
    im = cv2.resize(io.imread(image_file_name), (224, 224))
    im = im.transpose((2,0,1)) # convert the image to RGBA

    
    # this axis dimension is required because VGG was trained on a dimension
    # of 1, 3, 224, 224 (first axis is for the batch size
    # even though we are using only one image, we have to keep the dimensions consistent
    im = np.expand_dims(im, axis=0) 

    image_features[0,:] = model_vgg.predict(im)[0]
    return image_features

In [0]:
image_file_name = "http://www.newarkhistory.com/indparksoccerkids.jpg"
# get the image features
image_features = get_image_features(image_file_name)

Feel free to change that url to any valid image, it can be any image format. Also try to use websites which have higher bandwidth

<img src="http://www.newarkhistory.com/indparksoccerkids.jpg">
# <center> What are they playing? </center>

In [0]:
question = u"What are they playing?"

# get the question features
question_features = get_question_features(question)

In [0]:
y_output = model_vqa.predict([question_features, image_features])
warnings.filterwarnings("ignore", category=DeprecationWarning)
for label in reversed(np.argsort(y_output)[0,-5:]):
    print str(round(y_output[0,label]*100,2)).zfill(5), "% ", labelencoder.inverse_transform(label)

78.72 %  yes
21.28 %  no
000.0 %  girl
000.0 %  halloween
000.0 %  left


## Result
Copying the result to validate your output.

**55.44 %  frisbee<br/>
18.91 %  tennis<br/>
16.95 %  baseball<br/>
08.31 %  soccer<br/>
00.07 %  ball <br/>
**


As you can see, it got this wrong, but you can see why it could be harder to guess soccer and easier to guess frisbee, lack of soccer ball and lines at the edge.

Let's ask another question for the same image.

In [0]:
question = u"Are they playing Frisbee?"

# get the question features
question_features = get_question_features(question)

<img src="http://www.newarkhistory.com/indparksoccerkids.jpg">
# <center> Are they playing Frisbee? </center>

In [0]:
y_output = model_vqa.predict([question_features, image_features])

warnings.filterwarnings("ignore", category=DeprecationWarning)
for label in reversed(np.argsort(y_output)[0,-5:]):
    print str(round(y_output[0,label]*100,2)).zfill(5), "% ", labelencoder.inverse_transform(label)

78.72 %  yes
21.28 %  no
000.0 %  girl
000.0 %  halloween
000.0 %  left


## Result

**
78.72 %  yes <br />
21.28 %  no <br />
000.0 %  girl <br />
000.0 %  halloween <br />
000.0 %  left <br />
**

As you can see, similar information about a Yes/No question elicits different response. This is an impertinent problem with `classification` tasks.

Feel free to experiment with different types of questions, `count`, `color`, `location`.

More interesting results are obtained when one takes a different crop of a image, instead of just scaling it to 224x224. This is again because we extract only the top level features of CNN model which was trained to classify one object in the image.